In [3]:
from tensorflow.keras import layers
from tensorflow.keras import Sequential, optimizers
from tensorflow.keras.datasets import mnist
from PIL import Image
import numpy as np

In [23]:
(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))
x_train = x_train / 255.0
x_test = x_test / 255.0
y_train = np.copy(x_train)
y_test = np.copy(x_test)

x_train_noisy = np.clip(np.random.normal(loc=0, scale=0.5, size=(28, 28, 1)) + x_train, 0, 1)
x_test_noisy = np.clip(np.random.normal(loc=0, scale=0.5, size=(28, 28, 1)) + x_test, 0, 1)

In [24]:
def create_model(filters=[32, 64], learning_rate=0.001):
    model = Sequential()
    model.add(layers.InputLayer((28, 28, 1)))
    for filter_ in filters:
        model.add(layers.Conv2D(filter_, (3,3), padding="same", strides=2))
        model.add(layers.Activation("relu"))
        model.add(layers.BatchNormalization(axis=-1))        
    model.add(layers.Flatten())
    model.add(layers.Dense(16))
    
    model.add(layers.Dense(model.layers[-2].output_shape[1]))
    model.add(layers.Reshape(model.layers[-5].output_shape[1:]))
    for filter_ in filters[::-1]:
        model.add(layers.Conv2DTranspose(filter_, (3,3), padding="same", strides=2))
        model.add(layers.Activation("relu"))
        model.add(layers.BatchNormalization(axis=-1))
    
    model.add(layers.Conv2DTranspose(1, (3,3), padding="same"))
    model.add(layers.Activation("sigmoid"))
    
    opt = optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss="mse",optimizer=opt, metrics=["mse"])
    
    return model

In [25]:
model = create_model(filters=[64, 128])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 14, 14, 64)        640       
_________________________________________________________________
activation_17 (Activation)   (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
activation_18 (Activation)   (None, 7, 7, 128)         0         
_________________________________________________________________
batch_normalization_15 (Batc (None, 7, 7, 128)         512       
_________________________________________________________________
flatten_3 (Flatten)          (None, 6272)             

In [26]:
model.fit(x_train_noisy, y_train, batch_size=32, epochs=3)

Train on 60000 samples
Epoch 1/3
60000/60000 [==============================] - 27s 444us/sample - loss: 0.0196 - mse: 0.0196
Epoch 2/3
60000/60000 [==============================] - 26s 436us/sample - loss: 0.0098 - mse: 0.0098
Epoch 3/3
60000/60000 [==============================] - 26s 432us/sample - loss: 0.0088 - mse: 0.0088


In [27]:
cleaned_images = model.predict(x_test_noisy)

In [28]:
def concat_images(im1, im2, im3):
    dst = Image.new('L', (im1.width + im2.width + im3.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    dst.paste(im3, (im1.width+im2.width, 0))
    return dst

In [29]:
for i in range(100):
    normal_image = Image.fromarray((x_test[i]*255).astype(np.uint8).reshape((28, 28)))
    noisy_image = Image.fromarray((x_test_noisy[i]*255).astype(np.uint8).reshape((28, 28)))
    cleaned_image = Image.fromarray((cleaned_images[i]*255).astype(np.uint8).reshape((28, 28)))
    display(concat_images(normal_image, noisy_image, cleaned_image))